In [1]:
#some basic imports and setups
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from difflib import SequenceMatcher



#mean of imagenet dataset in BGR
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'memes')
#image_dir = current_dir

%matplotlib inline

In [ ]:
from alexnet import AlexNet

#placeholder for input and dropout rate
x = tf.placeholder(tf.float32, [1, 227, 227, 3])
keep_prob = tf.placeholder(tf.float32)

#create model with default config ( == no skip_layer and 1000 units in the last layer)
model = AlexNet(x, keep_prob, 1000,[],['fc7','fc8'],512,weights_path='/data/alpv95/MemeProject/im2txt/bvlc_alexnet.npy') #maybe need to put fc8 in skip_layers

#define activation of last layer as score
score = model.fc6

#create op to calculate softmax 
#softmax = tf.nn.softmax(score)

Converting captions and meme vector representations into single Tfrecord

Requires putting memes through alexnet to find their vector rep, shuffling the captions, changing captions into their word2idx, finally saving one caption together with one meme.

In [ ]:
#USE FOR ALEXNET !!!!!!!!!
#TRAINING SET

with open('/data/alpv95/MemeProject/im2txt/ordered_memes.txt','r') as f:
    img_files = f.readlines()
img_files = [os.path.join(image_dir, f) for f in img_files] # add path to each file
img_files = [img_file.replace('\n','') for img_file in img_files]
print(img_files[0],img_files[-1])
with open('/data/alpv95/MemeProject/im2txt/Captions.txt','r') as f:
    captions = f.readlines()
#captions = list(set(captions))
captions = [s.lower() for s in captions]
deleters = []
for i,capt in enumerate(captions):
    if ' - ' not in capt or ' - -' in capt:
        deleters.append(i)
for i,delete in enumerate(deleters):
    del captions[delete-i]
data_memes = []
data_captions = []
counter = 0
passed = 0


#Doing everything in one script: (the fc6 vectors are quite sparse)
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Load the pretrained weights into the model
    model.load_initial_weights(sess)
    
    for i,meme in enumerate(img_files):
        #meme_name = meme.replace('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/','')
        #meme_name = meme_name.replace('.jpg','').lower()
        #meme_name = meme_name.replace('-',' ')
        img = Image.open(meme)
        try:
            img.thumbnail((227, 227), Image.ANTIALIAS)
            #img = img.resize((227,227))
            #use img.thumbnail for square images, img.resize for non square
            assert np.shape(img) == (227, 227, 3)
        except AssertionError:
            img = img.resize((227,227))
            print('sizing error')
        
        # Subtract the ImageNet mean
        img = img - imagenet_mean #should probably change this
        
        # Reshape as needed to feed into model
        img = img.reshape((1,227,227,3))

        meme_vector = sess.run(score, feed_dict={x: img, keep_prob: 1}) #[1,4096]
        meme_vector = np.reshape(meme_vector,[4096])
        assert np.shape(meme_vector) == (4096,)
        #match = [s.split('-',1)[-1].lstrip() for s in captions if meme_name in s]
        
        match = []
        meme_name = captions[counter].split(' - ')[0]
        image_name = img_files[i].replace('/data/alpv95/MemeProject/im2txt/memes/','')
        image_name = image_name.replace('.jpg','')
        image_name = image_name.replace('-',' ')
        #print(meme_name,image_name)
        try:
            assert SequenceMatcher(a=meme_name.replace(' ',''),b=image_name.replace(' ','')).ratio() >= 0.75 or image_name in meme_name or meme_name in image_name
        except AssertionError:
            passed+=1
            continue
        
        while SequenceMatcher(a=meme_name.replace(' ',''),b=captions[counter].split(' - ')[0].replace(' ','')).ratio() >= 0.75 or meme_name in captions[counter].split(' - ')[0] or captions[counter].split(' - ')[0] in meme_name: 
            if counter==len(captions)-1:
                match.append(captions[counter].split(' - ')[-1])
                break
            elif captions[counter] == captions[counter].split(' - ')[-1]:
                counter += 1
            else:
                match.append(captions[counter].split(' - ')[-1])
                counter += 1
                
        
        #now save in tfrecords format, or prepare for that action
        meme_vectors = [meme_vector for cap in match]
        assert len(meme_vectors) == len(match)
        data_memes.extend(meme_vectors)
        data_captions.extend(match)

        if i % 100 == 0:
            print(i,len(data_memes),len(data_captions))

print(passed)
print(len(data_memes))
        
 

In [2]:
#USE FOR INCEPTION NETWORK (OR ALEXNET WITH FINETUNING)
#TRAINING SET
#For this case just need to save image filenames alongside captions
with open('/data/alpv95/MemeProject/im2txt/ordered_memes.txt','r') as f:
    img_files = f.readlines()
img_files = [os.path.join(image_dir, f) for f in img_files] # add path to each file
img_files = [img_file.replace('\n','') for img_file in img_files]
print(img_files[0],img_files[-1])
with open('/data/alpv95/MemeProject/im2txt/Captions.txt','r') as f:
    captions = f.readlines()
#captions = list(set(captions))
captions = [s.lower() for s in captions]
deleters = []
for i,capt in enumerate(captions):
    if ' - ' not in capt or ' - -' in capt:
        deleters.append(i)
for i,delete in enumerate(deleters):
    del captions[delete-i]
data_memes = []
data_captions = []
counter = 0
passed = 0


for i,meme in enumerate(img_files):
    
    match = []
    meme_name = captions[counter].split(' - ')[0]
    image_name = meme.replace('/data/alpv95/MemeProject/im2txt/memes/','')
    image_name = image_name.replace('.jpg','')
    image_name = image_name.replace('-',' ')
    #print(meme_name,image_name)
    try:
        assert SequenceMatcher(a=meme_name.replace(' ',''),b=image_name.replace(' ','')).ratio() >= 0.75 or image_name in meme_name or meme_name in image_name
    except AssertionError:
        passed+=1
        continue
        
    while SequenceMatcher(a=meme_name.replace(' ',''),b=captions[counter].split(' - ')[0].replace(' ','')).ratio() >= 0.75 or meme_name in captions[counter].split(' - ')[0] or captions[counter].split(' - ')[0] in meme_name: 
        if counter==len(captions)-1:
            match.append(captions[counter].split(' - ')[-1])
            break
        elif captions[counter] == captions[counter].split(' - ')[-1]:
            counter += 1
        else:
            match.append(captions[counter].split(' - ')[-1])
            counter += 1
                
        
    #now save in tfrecords format, or prepare for that action
    meme_images = [meme for cap in match]
    assert len(meme_images) == len(match)
    data_memes.extend(meme_images)
    data_captions.extend(match)

    if i % 100 == 0:
        print(i,len(data_memes),len(data_captions))
print(passed)
print(len(data_memes))
        

('/data/alpv95/MemeProject/im2txt/memes/y-u-no.jpg', '/data/alpv95/MemeProject/im2txt/memes/mendozameme.jpg')
(0, 194, 194)
(100, 19500, 19500)
(200, 38642, 38642)
(300, 57715, 57715)
(400, 76329, 76329)
(500, 94591, 94591)
(600, 113146, 113146)
(700, 131507, 131507)
(800, 149520, 149520)
(900, 167518, 167518)
(1000, 185425, 185425)
(1100, 203294, 203294)
(1200, 220336, 220336)
(1300, 237823, 237823)
(1400, 254129, 254129)
(1500, 270058, 270058)
(1600, 286622, 286622)
(1700, 302200, 302200)
(1800, 316652, 316652)
(1900, 331886, 331886)
(2000, 346319, 346319)
(2100, 362089, 362089)
(2200, 375126, 375126)
(2300, 389123, 389123)
(2400, 401660, 401660)
(2500, 414389, 414389)
23
414389


In [ ]:
for i in range(200):
    print(data_memes[i+402000])
    print(data_captions[i+402000])

In [ ]:
data_memes[len(data_memes)-1]

In [ ]:
#CREATE EVALUATION SET
#SHOULD BE MEMES WITH REPEATED FORMAT 
for i in range(100,200):
    print(data_memes[i*190].replace('/data/alpv95/MemeProject/im2txt/memes/',''),data_captions[i*190],i*190)

In [3]:
eval_examples = [0,570,760,1140,2850,3040,4940,28500]
eval_captions = ['y u no','i dont always','not sure if','one does not simply','what if i told you','what if','so youre telling me','so then i said']
eval_memes = []

for idx in eval_examples:
    eval_memes.append(data_memes[idx])


In [4]:
#Should also look at removing examples which are similar but not exactly the same

c = list(zip(data_memes, data_captions))
no_repeats = []
# order preserving
def idfun(x): return x

seen = {}
no_repeats = []
for item in c:
    marker = idfun(item[1])
    # in old Python versions:
    # if seen.has_key(marker)
    # but in new ones:
    if marker in seen: continue
    seen[marker] = 1
    no_repeats.append(item)
print(len(data_captions))
len(no_repeats)


414389


385708

In [5]:
from random import shuffle
shuffle(no_repeats)
memes_shuffled, captions_shuffled = zip(*no_repeats)

In [6]:
import re
word_captions = []
for capt in captions_shuffled:
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    word_captions.append(words)
print(len(word_captions))
#word_captions = list(set(word_captions))
#print(len(word_captions))

385708


In [7]:
from collections import Counter
print("Creating vocabulary.")
counter = Counter()
for c in word_captions:
    counter.update(c)
print("Total words:", len(counter))

# Filter uncommon words and sort by descending count.
word_counts = [x for x in counter.items() if x[1] >= 3]
word_counts.sort(key=lambda x: x[1], reverse=True)
print("Words in vocabulary:", len(word_counts))

Creating vocabulary.
('Total words:', 150280)
('Words in vocabulary:', 41153)


In [8]:
# Create the vocabulary dictionary.
reverse_vocab = [x[0] for x in word_counts]
#unk_id = len(reverse_vocab)
vocab_dict = dict([(x, y) for (y, x) in enumerate(reverse_vocab)])

In [9]:
vocab_dict['*']

81

In [10]:
EMBEDDING_DIMENSION=300 # Available dimensions for 6B data is 50, 100, 200, 300
data_directory = current_dir

PAD_TOKEN = 0

word2idx = { 'PAD': PAD_TOKEN } # dict so we can lookup indices for tokenising our text later from string to sequence of integers
weights = []
index_counter = 0

with open('glove.42B.300d.txt','r') as file:
    for index, line in enumerate(file):
        values = line.split() # Word and weights separated by space
        word = values[0] # Word is first symbol on each line
        if word in vocab_dict:
            index_counter += 1
            word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
            word2idx[word] = index_counter # PAD is our zeroth index so shift by one
            weights.append(word_weights)
        if index % 100000 == 0:
            print(index)
        if index + 1 == 2000000:
            break

EMBEDDING_DIMENSION = len(weights[0])
# Insert the PAD weights at index 0 now we know the embedding dimension
weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))

# Append unknown and pad to end of vocab and initialize as random #maybe include start and end token here
UNKNOWN_TOKEN=len(weights)
#include * token as its very common? why not
word2idx['*'] = UNKNOWN_TOKEN
word2idx['UNK'] = UNKNOWN_TOKEN + 1
word2idx['<S>'] = UNKNOWN_TOKEN + 2
word2idx['</S>'] = UNKNOWN_TOKEN + 3
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)

# Construct our final vocab
weights = np.asarray(weights, dtype=np.float32)

VOCAB_SIZE=weights.shape[0]
print(VOCAB_SIZE)

#Save Vocabulary
with tf.gfile.FastGFile('vocab4.txt', "w") as f:
    f.write("\n".join(["%s %d" % (w, c) for w, c in word2idx.iteritems()]))
print("Wrote vocabulary file:", 'vocab4.txt')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
38521
('Wrote vocabulary file:', 'vocab4.txt')


In [ ]:
weights[37984]

In [11]:
np.savetxt('embedding_matrix5',weights)

In [12]:
import re
token_captions = []
for capt in captions_shuffled:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_caption.append(word2idx['</S>'])
    token_captions.append(token_caption)
    
#for eval set
eval_tokens = []
for capt in eval_captions:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    #token_caption.append(word2idx['</S>'])
    eval_tokens.append(token_caption)

In [13]:
print(token_captions[330000])
print(captions_shuffled[330000])
print(eval_tokens[3])
print(eval_captions[3])
word2idx['UNK']

[38519, 70, 2150, 38518, 24633, 33, 38520]
no le copio mami! 

[38519, 46, 137, 31, 716]
one does not simply


38518

In [ ]:
type(eval_captions)

In [14]:
memes_shuffled = list(memes_shuffled)
captions_shuffled = list(captions_shuffled)

In [15]:
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)

In [16]:
for i,ting in enumerate(deleters):
    print(ting)
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)
len(deleters)

62080
95509
128068
137995
143522
158280
158395
161500
192054
194952
311134
365078


0

In [17]:
many_unk = []
for i,capt in enumerate(token_captions):
    unk_counter = 0
    for token in capt:
        if token == word2idx['UNK']:
            unk_counter += 1
    if unk_counter >= 2:
        many_unk.append(i)
print(len(many_unk))

for i,ting in enumerate(many_unk):
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]

30259


In [18]:
len(memes_shuffled)

355437

In [19]:

def _int64_feature(value):
    """Wrapper for inserting an int64 Feature into a SequenceExample proto."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    """Wrapper for inserting a bytes Feature into a SequenceExample proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature_list(values):
    """Wrapper for inserting an int64 FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_int64_feature(v) for v in values])


def _bytes_feature_list(values):
    """Wrapper for inserting a bytes FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_bytes_feature(v) for v in values])

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [ ]:
#ONLY FOR ALEXNET 
memes_shuffled_int = []
for i,meme in enumerate(memes_shuffled):
    memes_shuffled_int.append(np.int_(meme*1000000000000000))
print(memes_shuffled_int[20][:100])

In [20]:
#ONLY FOR INCEPTION
class ImageDecoder(object):
    """Helper class for decoding images in TensorFlow."""

    def __init__(self):
        # Create a single TensorFlow Session for all image decoding calls.
        self._sess = tf.Session()

        # TensorFlow ops for JPEG decoding.
        self._encoded_jpeg = tf.placeholder(dtype=tf.string)
        self._decode_jpeg = tf.image.decode_jpeg(self._encoded_jpeg, channels=3)

    def decode_jpeg(self, encoded_jpeg):
        image = self._sess.run(self._decode_jpeg,
                               feed_dict={self._encoded_jpeg: encoded_jpeg})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

In [ ]:
#SAVING TRAINING SET (ALEXNET)
import sys
train_filename = 'train.tfrecords6'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled_int)):
    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled_int))
        sys.stdout.flush()
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(memes_shuffled_int[i].tostring()), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(token_captions[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

In [ ]:
#SAVING TRAINING SET (INCEPTION)
import sys
decoder = ImageDecoder()
train_filename = 'train.tfrecords7'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled)):
    
    with tf.gfile.FastGFile(memes_shuffled[i], "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        break

    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled))
        sys.stdout.flush()
    
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(encoded_image), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(token_captions[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/355437
Train data: 20000/355437
Train data: 40000/355437
Train data: 60000/355437
Train data: 80000/355437
Train data: 100000/355437
Train data: 120000/355437
Train data: 140000/355437
Train data: 160000/355437
Train data: 180000/355437
Train data: 200000/355437
Train data: 220000/355437
Train data: 240000/355437
Train data: 260000/355437
Train data: 280000/355437


In [ ]:
#SAVING EVAL SET (INCEPTION)
import sys
decoder = ImageDecoder()
train_filename = 'eval.tfrecords7'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(eval_memes)):
    
    with tf.gfile.FastGFile(eval_memes[i], "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        break

    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(eval_memes))
        sys.stdout.flush()
    
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(encoded_image), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(eval_tokens[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

In [ ]:
eval_tokens[2]

In [ ]:
x

In [ ]:
for i,wp in enumerate(x):
    if wp[0] == 1:
        del x[i]
x = x[0:2]
print(x)

In [ ]:
with tf.gfile.GFile('vocab2.txt', mode="r") as f:
      wordidx_pairs = list(f.readlines())
wordidx_pairs = [(line.split()[0],line.split()[1]) for line in wordidx_pairs]
vocab = dict([(x, int(y)) for (x, y) in wordidx_pairs]) #changed this to reflect vocab.txt format
x = sorted(vocab.iteritems(), key=lambda x: int(x[1]))
reverse_vocab = [y[0] for y in x]


In [ ]:
type(vocab[','])